In [41]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import xgboost as xgb 
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
from imblearn.over_sampling import SMOTE

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return hour
def make_datetime_day(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return day

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)



PATH = 'C:/Users/hyeonggu/Desktop/lg 챌린지/data (2)/'

In [42]:
import pandas as pd

# 최대 줄 수 설정

pd.set_option('display.max_rows', 500)
# 최대 열 수 설정
pd.set_option('display.max_columns', 500)
# 표시할 가로의 길이
pd.set_option('display.width', 1000)

In [43]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [4]:
qdata  = pd.read_csv(PATH+'train_quality_data.csv')
t_qdata  = pd.read_csv(PATH+'test_quality_data.csv')

In [5]:
train_err  = pd.read_csv(PATH+'both_train.csv')
len(train_err['pro_err'].unique())

1493

In [6]:
train_err["hour"] = train_err['time'].apply(make_datetime)
train_err["day"] = train_err['time'].apply(make_datetime_day)

#hour = [3,4,5]
#train_err = train_err[~train_err['hour'].isin(hour)]

In [7]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

train_prob["hour"] = train_prob['time'].apply(make_datetime)
count = dict(train_prob['hour'].value_counts())

In [8]:
count = dict(train_prob['hour'].value_counts())

In [9]:
train_err

,Unnamed: 0,Unnamed: 0.1,user_id,time,model_nm,fwver,errtype,errcode,pro_err,both_err,pro_both,hour,day
0,0,0,10000,20201101025616,model_3,05.15.2138,15,1,15.1,15.1,15,2,1
1,1,1,10000,20201101030309,model_3,05.15.2138,12,1,12.1,12.1,10,3,1
2,2,2,10000,20201101030309,model_3,05.15.2138,11,1,11.1,11.1,9,3,1
3,3,3,10000,20201101050514,model_3,05.15.2138,16,1,16.1,16.1,16,5,1
4,4,4,10000,20201101050515,model_3,05.15.2138,4,0,4.0,4.0,1401,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16554658,16554658,16554658,24999,20201130163051,model_3,05.15.2138,15,1,15.1,15.1,15,16,30
16554659,16554659,16554659,24999,20201130172625,model_3,05.15.2138,16,1,16.1,16.1,16,17,30
16554660,16554660,16554660,24999,20201130172625,model_3,05.15.2138,4,0,4.0,4.0,1401,17,30
16554661,16554661,16554661,24999,20201130172631,model_3,05.15.2138,4,0,4.0,4.0,1401,17,30


In [10]:
for i in range(24):
    if 590 < count[i]:
        count[i]= 1.3
    elif 500<=count[i] and count[i] <590:
        count[i] = 1.3
    elif 400<=count[i] and count[i] <500:
        count[i] = 1.2
    elif 200<=count[i] and count[i] <400:
        count[i] = 1.1
    else:
        count[i]= 1


In [11]:
count

{12: 1.3,
 11: 1.3,
 13: 1.3,
 19: 1.3,
 18: 1.2,
 15: 1.2,
 16: 1.2,
 17: 1.2,
 22: 1.1,
 20: 1.1,
 14: 1.1,
 21: 1.1,
 23: 1,
 0: 1,
 1: 1,
 2: 1,
 10: 1,
 9: 1,
 5: 1,
 8: 1,
 3: 1,
 4: 1,
 6: 1,
 7: 1}

In [12]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','pro_both','hour']].values
error = np.zeros((train_user_number,1493))

for person_idx, err,hour in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += count[hour]
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:53<00:00, 307317.40it/s]


(15000, 1493)

In [13]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [99]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = error
train_y = problem
#del error, problem
print(train_x.shape)
print(train_y.shape)

(15000, 1493)
(15000,)


In [100]:
user_df = pd.read_csv("user_df2.csv")
user_df3 = pd.read_csv("user_df3.csv")

user_df.head()
time_count = pd.read_csv("time_count.csv")


In [101]:
tem = pd.DataFrame(train_x)
#tem["user_model"] = user_df['user_model']
#tem["user_quality"] = user_df['quality']
#tem["user_fwver"] = user_df['user_fwver']
#tem['change_model'] = user_df['change_model']
#tem['change_fwver'] = user_df['change_fvwer']
#tem['most_type'] = user_df['most_type']
#tem['change_model'] = user_df['c']
#tem['q_time_count'] = time_count['q_time_count']
#tem['err_time_count'] = time_count['err_time_count']
#tem['user_fwver_count'] = time_count['user_fwver']

#tem["user_fwver"] = user_df3['user_fwver']
#tem['change_fwver'] = user_df3['change_fwver']
train_x = tem.values
train_x.shape

(15000, 1493)

In [ ]:
pro_qu  = pd.read_csv(PATH+'pro_quality.csv')

In [ ]:
tem = pd.DataFrame(train_x)
tem["q_a"] = pro_qu['quality_average']
tem["q_l"] = pro_qu['quality_last']
train_x = tem.values

train_x.shape

In [102]:
q= pd.read_csv(PATH+'q3.csv')
#fwver_quality.csv
#q= pd.read_csv(PATH+'q.csv')
train_x  = np.concatenate([train_x,q.values],axis = 1)
train_x.shape
tem = pd.DataFrame(train_x)
#tem = tem.fillna(0)
train_x = tem.values
train_x.shape

(15000, 1502)

In [103]:
#에러타입 누적합
sum31 = pd.read_csv(PATH+'sum31.csv')
sum31.head()
train_x  = np.concatenate([train_x,sum31.values],axis = 1)
train_x.shape

(15000, 1533)

In [104]:
#에러타입 표준편차
std = pd.read_csv(PATH+'std.csv')
train_x  = np.concatenate([train_x,std.values],axis = 1)
train_x.shape

(15000, 1534)

q_real = pd.read_csv(PATH+'q_real.csv')
#del q_real['4']

del q_real['3']
del q_real['2']
del q_real['1']
del q_real['0']

train_x  = np.concatenate([train_x,q_real.values],axis = 1)
train_x.shape

In [105]:
q_real.head()

,4
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [106]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings

In [694]:
train_x = pd.DataFrame(data=train_x)
train_x['problem'] = pd.Series(train_y)
clf = setup(data = train_x, target = 'problem')

,Description,Value
0,session_id,4399
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 1541)"
5,Missing Values,True
6,Numeric Features,864
7,Categorical Features,676
8,Ordinal Features,False
9,High Cardinality Features,False


In [695]:
#include=['catboost','lightgbm','gbc','et','rf','ada','xgboost'],
#exclude=['lr','knn','qda']

In [696]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7944,0.8276,0.5137,0.7999,0.6255,0.4929,0.5159,10.9890
lightgbm,Light Gradient Boosting Machine,0.7942,0.8260,0.5308,0.7844,0.6330,0.4972,0.5157,0.2960
gbc,Gradient Boosting Classifier,0.7941,0.8209,0.4906,0.8221,0.6143,0.4858,0.5160,2.3950
et,Extra Trees Classifier,0.7872,0.8146,0.4766,0.8086,0.5993,0.4674,0.4979,2.1100
ada,Ada Boost Classifier,0.7862,0.8094,0.5071,0.7756,0.6132,0.4743,0.4947,0.6980
xgboost,Extreme Gradient Boosting,0.7851,0.8141,0.5430,0.7456,0.6282,0.4823,0.4944,5.9800
rf,Random Forest Classifier,0.7836,0.8141,0.4900,0.7816,0.6022,0.4641,0.4881,1.0050
ridge,Ridge Classifier,0.7610,0.0000,0.3920,0.7866,0.5229,0.3865,0.4284,0.2300
lda,Linear Discriminant Analysis,0.7570,0.7423,0.3980,0.7621,0.5225,0.3805,0.4168,2.2090
dt,Decision Tree Classifier,0.7028,0.6722,0.5798,0.5533,0.5661,0.3404,0.3407,0.1780


In [697]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')
#catboost = create_model('catboost')
#stacker = stack_models(estimator_list = best_5[1:], meta_model = best_5[0])


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7905,0.8172,0.5100,0.7885,0.6194,0.4839,0.5057
1,0.7914,0.8117,0.5071,0.7946,0.6191,0.4850,0.5082
2,0.8143,0.8547,0.5299,0.8611,0.6561,0.5386,0.5683
3,0.8038,0.8357,0.5556,0.7959,0.6544,0.5234,0.5399
4,0.7848,0.8165,0.5157,0.7637,0.6156,0.4739,0.4915
5,0.8086,0.8442,0.5242,0.8440,0.6467,0.5251,0.5531
6,0.8086,0.8480,0.5271,0.8409,0.6480,0.5258,0.5529
7,0.7800,0.8347,0.4815,0.7752,0.5940,0.4542,0.4785
8,0.7895,0.8045,0.5214,0.7754,0.6235,0.4851,0.5035
9,0.7884,0.8144,0.4986,0.7919,0.6119,0.4765,0.5006


In [698]:
#pred_holdout = predict_model(stacker)
#pred_holdout = predict_model(catboost)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.8034,0.8346,0.5409,0.8004,0.6456,0.5165,0.5355


In [699]:

final_model = finalize_model(blended)
#final_model = finalize_model(catboost)
#final_model = finalize_model(stacker)


In [700]:
test_x = pd.DataFrame(test_x)

In [701]:
predictions = predict_model(final_model, data = test_x)

In [702]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])

In [703]:
save_model(final_model, 's_dt_saved_8346')


Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='problem',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_stra...
                                                                       n_estimators=100,
                                                                       n_jobs=-1,
                                                                       oob_score=False,
               

In [704]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv("s_AutoML(8346).csv", index = False)

In [662]:
sample_submssion.head()

,user_id,problem
0,30000,0.9401
1,30001,0.2186
2,30002,0.4291
3,30003,0.8194
4,30004,0.9028


In [663]:
a = pd.read_csv("dacon_baseline_8299.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'dacon_baseline_8299.csv'

In [107]:
train_x = pd.DataFrame(train_x)
go = pd.read_csv('total_try2.csv')
train_x  = np.concatenate([train_x,go.values],axis = 1)
train_x.shape

(15000, 1566)

In [108]:
q3= pd.read_csv(PATH+'506.csv')
train_x  = np.concatenate([train_x,q3.values],axis = 1)
train_x.shape

tem = pd.DataFrame(train_x)
#tem = tem.fillna(0)
train_x = tem.values
train_x.shape


(15000, 2072)

In [109]:

#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정



params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }


#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=13)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traningd
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 200
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 4471, number of negative: 9029
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030513 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71093
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 606
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.331185 -> initscore=-0.702830
[LightGBM] [Info] Start training from score -0.702830
Training until validation scores don't improve for 200 rounds
[20]	valid_0's auc: 0.829313	valid_0's pr_auc: 0.779571
[40]	valid_0's auc: 0.834181	valid_0's pr_auc: 0.786873
[60]	valid_0's auc: 0.834186	valid_0's pr_auc: 0.783319
[80]	valid_0's auc: 0.832985	valid_0's pr_auc: 0.784928
[100]	valid_0's auc: 0.833376	valid_0's pr_auc: 0.786547
[120]	valid_0's auc: 0.833975	valid_0's pr_auc: 0.787816
[140]	valid_0's auc: 0.833641	valid_0's pr_auc: 0.786267
[160]	valid_0's auc: 0.831955	valid_0's pr_auc

[LightGBM] [Info] Number of positive: 4493, number of negative: 9007
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71055
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 606
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.332815 -> initscore=-0.695481
[LightGBM] [Info] Start training from score -0.695481
Training until validation scores don't improve for 200 rounds
[20]	valid_0's auc: 0.807435	valid_0's pr_auc: 0.733594
[40]	valid_0's auc: 0.813537	valid_0's pr_auc: 0.735264
[60]	valid_0's auc: 0.812665	valid_0's pr_auc: 0.730974
[80]	valid_0's auc: 0.813124	valid_0's pr_auc: 0.733477
[100]	valid_0's auc: 0.81656	valid_0's pr_auc: 0.734457
[120]	valid_0's auc: 0.81515	valid_0's pr_auc: 0.733818
[140]	valid_0's auc: 0.814915	valid_0's pr_auc: 0.731578
[160]	valid_0's auc: 0.813966	valid_0's pr_auc: 

In [110]:
print(np.mean(auc_scores))

0.8365982941904211


test_err  = pd.read_csv(PATH+'both_test.csv')
len(train_err['pro_err'].unique())

In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [687]:
test_err["hour"] = test_err['time'].apply(make_datetime)

In [688]:
id_error = test_err[['user_id','pro_both','hour']].values
test_x = np.zeros((test_user_number,1493))
for person_idx, err,hour in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += count[hour]
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:53<00:00, 310149.34it/s]

(14999, 1493)


In [689]:
test_x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [690]:
t_user_df = pd.read_csv("t_user_df2.csv")
tem = pd.DataFrame(test_x)
tem["user_model"] = t_user_df['user_model']
tem["user_quality"] = t_user_df['quality']
tem["user_fwver"] = t_user_df['user_fwver']
tem["model_change"] = t_user_df['change_model']
test_x = tem.values
test_x.shape

(14999, 1497)

t_pro_qu = pd.read_csv('test_quality.csv')
tem = pd.DataFrame(test_x)
tem["q_a"] = t_pro_qu['quality_average']
tem["q_l"] = t_pro_qu['quality_last']
test_x = tem.values
test_x.shape

In [691]:
#퀄리티 데이터 통째로 
t_q = pd.read_csv('t_q3.csv')
test_x  = np.concatenate([test_x,t_q.values],axis = 1)
test_x.shape

(14999, 1506)

In [692]:
#에러타입 누적합
t_sum31 = pd.read_csv(PATH+'t_sum31.csv')
test_x  = np.concatenate([test_x,t_sum31.values],axis = 1)
test_x.shape

#에러타입 누적합

t_std = pd.read_csv(PATH+'t_std.csv')
test_x  = np.concatenate([test_x,t_std.values],axis = 1)

#퀄리티 데이터 누적합
t_ql = pd.read_csv(PATH+'test_null_count.csv')
del t_ql['quality_0']
test_x  = np.concatenate([test_x,t_ql.values],axis = 1)
test_x.shape

In [693]:
#퀄리티 max
t_q = pd.read_csv(PATH+'test_quality4.csv')
gg = ['max_quality_10','max_quality_12']

ttt = t_q[gg]
test_x  = np.concatenate([test_x,ttt.values],axis = 1)
test_x.shape

(14999, 1540)

t_q_real = pd.read_csv(PATH+'t_q10_real.csv')
#del q_real['4']

del t_q_real['3']
del t_q_real['2']
del t_q_real['1']
del t_q_real['0']

test_x  = np.concatenate([test_x,t_q_real.values],axis = 1)
test_x.shape

test_common = pd.read_csv("test_total2.csv")
del test_common['usr_id']

test_x  = np.concatenate([test_x,test_common.values],axis = 1)
test_x.shape


In [86]:
#test_x = np.delete(test_x,42,axis = 1)
#test_x = np.delete(test_x,42,axis = 1)
#test_x = np.delete(test_x,42,axis = 1)
#test_x.shape

# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis 
                        = 0)
pred_ensemble

In [1181]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


In [1182]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [1183]:
sample_submssion.to_csv("dacon_baseline_8322.csv",index =False)

In [1184]:
t_q = pd.read_csv(PATH+'test_quality4.csv')

In [1185]:
t_q['max_quality_10'].value_counts()

0.0       7008
3.0       1262
2.0        846
4.0        740
5.0        538
          ... 
2964.0       1
9114.0       1
7983.0       1
5430.0       1
1287.0       1
Name: max_quality_10, Length: 1284, dtype: int64

In [1186]:
t['max_quality_10'].value_counts()

0.0        7023
3.0        1277
2.0         889
4.0         748
5.0         501
           ... 
4559.0        1
1710.0        1
1027.0        1
781.0         1
13691.0       1
Name: max_quality_10, Length: 1323, dtype: int64

In [240]:
test_x.shape

(14999, 1542)

In [259]:
test_x = pd.DataFrame(test_x)

In [260]:
test_x.head()

,0,1,2,3,4,5,6,7,8,9,...,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,...,2343.0,2528.0,2686.0,2711.0,2724.0,2745.0,2750.0,148.649011,2.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,16.8,...,250.0,271.0,273.0,275.0,275.0,280.0,284.0,6.421219,2556.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.2,16.2,...,699.0,732.0,799.0,828.0,870.0,900.0,941.0,14.353933,49.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.9,6.1,...,270.0,311.0,319.0,322.0,338.0,348.0,371.0,14.787710,6.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,12.1,...,623.0,653.0,715.0,766.0,812.0,857.0,881.0,16.954920,2.0,0.0


In [261]:
train_x.head()

,0,1,2,3,4,5,6,7,8,9,...,1534,1535,1536,1537,1538,1539,1540,1541,1542,problem
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.1,16.1,17.1,...,265.0,277.0,295.0,308.0,317.0,5.001353,8.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.2,11.2,...,785.0,2237.0,2275.0,2337.0,2365.0,251.635963,0.0,0.0,NaN,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3,13.3,14.6,...,273.0,289.0,289.0,297.0,306.0,4.513363,8.0,0.0,44.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.2,9.2,...,254.0,275.0,286.0,300.0,306.0,6.178746,0.0,0.0,NaN,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,19.0,...,690.0,714.0,732.0,755.0,777.0,17.479823,3.0,0.0,48.0,1.0


In [138]:
q3.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
count,14484.0,15000.000000,15000.000000,1.500000e+04,15000.000000,14995.000000,14510.000000,1.491200e+04,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,14998.000000,15000.000000,15000.000000,1.498900e+04,13609.000000,14782.000000,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.499300e+04,15000.000000,14977.000000,12778.000000,14633.0,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,1.499700e+04,15000.000000,14983.000000,12753.000000,14619.0,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,14992.000000,14997.0,15000.000000,15000.000000,15000.0,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.00000,14975.0,14997.0,15000.000000,15000.000000,15000.0,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000,1.498700e+04,13339.000000,1.472200e+04,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,14999.000000,14989.0,14998.0,15000.000000,15000.000000,15000.0,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,14999.000000,14977.000000,14993.000000,1.500000e+04,15000.000000,15000.000000,14941.0,15000.000000,14941.0,15000.000000,15000.000000,14999.000000,14944.000000,14988.0,1.500000e+04,15000.000000,15000.000000
mean,0.0,-0.756600,59.732333,2.229682e+02,5.749667,79.865822,1.094142,7.318012e+02,2.686470e+03,-0.740800,1.961600,-1.315400,-1.201133,-1.310775,372.849067,25.108067,3.160710e+02,0.025571,9.842241,4.489974e+03,-1.314133,0.014400,-2.582867,-2.459333,-2.582867,7.269767e+02,10.998600,162.948254,0.087338,0.0,1.924126e+04,-2.576533,0.076000,-2.747133,-2.575400,-2.747133,7.884529e+02,18.704600,257.704065,0.063044,0.0,1.139143e+04,-2.722733,0.292800,-0.002867,-0.002467,-0.002867,1.226133,0.000533,0.040800,0.000800,0.0,14.748000,-0.002867,0.0,-0.020933,-0.019667,-0.020933,0.506533,0.216267,2.84640,0.0,0.0,6.329600,-0.020933,0.0,194.907333,-1.846667,194.885533,9.320281e+02,50.087733,6.245885e+02,0.019792,2.406308e+03,1.120798e+04,-1.955867,0.177600,-0.011333,-0.010800,-0.011333,0.051400,0.027133,0.461631,0.0,0.0,0.752800,-0.011333,0.0,1.007067,-0.055333,1.007067,31.036267,0.060267,1.571305,0.000801,12.938838,3.732832e+02,-0.070133,0.006400,0.0,-0.562133,0.0,9.172533,1.927333,30.437229,0.002409,0.0,1.172408e+02,-0.618667,0.005600
std,0.0,15.484405,4563.897465,2.037203e+04,352.359594,4227.940921,131.299693,5.493456e+04,2.444825e+05,10.335076,35.429872,14.288646,13.973373,14.283964,10835.373883,1927.995704,2.314418e+04,1.053782,1196.631896,1.300676e+05,14.284462,0.619531,22.293473,22.160106,22.293473,1.774400e+04,270.383618,3260.531774,4.042958,0.0,5.942285e+05,22.268016,2.897375,23.130700,22.851312,23.130700,1.858445e+04,523.224879,6289.160621,4.105545,0.0,2.641190e+05,22.929299,14.414164,0.177183,0.148533,0.177183,93.791754,0.452563,4.996959,0.098006,0.0,1126.014224,0.177183,0.0,1.060835,1.003335,1.060835,59.753396,21.155677,255.30527,0.0,0.0,727.134508,1.060835,0.0,8501.936510,20.747767,8501.933091,3.400109e+04,2561.714163,3.075689e+04,1.038860,1.030084e+05,4.081537e+05,20.806688,6.078661,0.749161,0.730515,0.749161,3.252158,3.490063,42.286751,0.0,0.0,42.933709,0.749161,0.0,123.009580,2.433242,123.009580,3522.416266,6.329632,111.806624,0.098055,1485.336653,4.228890e+04,3.544994,0.783837,0.0,16.792491,0.0,737.245331,162.539229,1995.679271,0.219493,0.0,8.887705e+03,17.607356,0.509103
min,0.0,-580.000000,-580.000000,-2.090000e+02,-580.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-580.000000,0.000000,-832.000000,-811.000000,-832.000000,-146.000000,-832.000000,0.000000e+00

In [134]:
q.head(500)

,0,1,2,3,4,5,6,7,8
0,0.0,0.0,12.0,0.0,0.0,0.0,144.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.0,-1.0,29.0,44.0,552.0,12.0,372.0,-2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.0,-2.0,2.0,85.0,1044.0,0.0,48.0,-2.0,0.0
5,-10.0,-10.0,0.0,26.0,432.0,0.0,120.0,-10.0,0.0
6,0.0,0.0,12.0,4.0,48.0,0.0,144.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [244]:
qdata.groupby(by=['user_id','fwver']).count()

,,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
user_id,fwver,,,,,,,,,,,,,,
10000,05.15.2138,24,24,24,24,24,24,24,24,24,24,24,24,24,24
10002,05.15.2138,96,96,96,96,96,96,96,96,96,96,96,96,96,96
10004,04.22.1750,24,24,24,24,24,24,24,24,24,24,24,24,24,24
10005,04.22.1750,24,24,24,24,24,24,24,24,24,24,24,24,24,24
10006,04.16.3553,12,12,12,12,12,12,12,12,12,12,12,12,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24992,04.22.1750,36,36,36,36,36,36,36,36,36,36,36,36,36,36
24993,05.15.2138,24,24,24,24,24,24,24,24,24,24,24,24,24,24
24995,04.33.1261,24,24,24,24,24,24,24,24,24,24,24,24,24,24


In [254]:
fwver_num = dict()
q = qdata['fwver'].unique()
for i in q:
    tem =  len(qdata[qdata['fwver']==i]['user_id'].unique())
    fwver_num[str(i)] = tem 
    

In [256]:
fwver_num

{'05.15.2138': 1724,
 '04.22.1750': 1967,
 '04.16.3553': 1443,
 '04.33.1261': 1952,
 '04.22.1778': 1101,
 '04.33.1185': 1159,
 '04.16.3571': 68,
 '05.66.3571': 5,
 '03.11.1149': 2,
 '03.11.1167': 514,
 '04.82.1684': 22,
 '04.82.1778': 13,
 '04.33.1149': 37,
 'nan': 0,
 '05.66.3237': 8,
 '04.73.2237': 8,
 '09.17.1431': 25,
 '04.22.1684': 22,
 '05.15.2120': 4,
 '04.33.1125': 3,
 '05.15.2122': 1,
 '04.22.1666': 1,
 '04.22.1656': 1,
 '04.16.3439': 1,
 '04.73.2571': 1,
 '05.15.2114': 1,
 '04.16.3345': 1,
 '04.22.1442': 1}